In [32]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt


In [18]:
df = pd.read_csv("eye_movements.csv")
df.head

<bound method NDFrame.head of        lineNo  assgNo  fixcount  firstPassCnt  P1stFixation  P2stFixation  \
0           1       1         1             1             1             0   
1           2       1         1             1             1             0   
2           3       1         1             1             1             0   
3           4       1         1             1             1             0   
4           5       1         1             1             1             0   
...       ...     ...       ...           ...           ...           ...   
10931   10932     336         1             1             1             0   
10932   10933     336         1             1             1             0   
10933   10934     336         1             1             1             0   
10934   10935     336         2             1             1             0   
10935   10936     336         1             1             1             1   

       prevFixDur  firstfixDur  firstPassFixD

In [19]:
num_missing_values = df.isna().sum()
num_missing_values # No need to remove any tuples or perform data imputation since none of the data is missing

lineNo             0
assgNo             0
fixcount           0
firstPassCnt       0
P1stFixation       0
P2stFixation       0
prevFixDur         0
firstfixDur        0
firstPassFixDur    0
nextFixDur         0
firstSaccLen       0
lastSaccLen        0
prevFixPos         0
landingPos         0
leavingPos         0
totalFixDur        0
meanFixDur         0
nRegressFrom       0
regressLen         0
nextWordRegress    0
regressDur         0
pupilDiamMax       0
pupilDiamLag       0
timePrtctg         0
nWordsInTitle      0
titleNo            0
wordNo             0
label              0
dtype: int64

In [20]:
attributes = df.columns
attributes

Index(['lineNo', 'assgNo', 'fixcount', 'firstPassCnt', 'P1stFixation',
       'P2stFixation', 'prevFixDur', 'firstfixDur', 'firstPassFixDur',
       'nextFixDur', 'firstSaccLen', 'lastSaccLen', 'prevFixPos', 'landingPos',
       'leavingPos', 'totalFixDur', 'meanFixDur', 'nRegressFrom', 'regressLen',
       'nextWordRegress', 'regressDur', 'pupilDiamMax', 'pupilDiamLag',
       'timePrtctg', 'nWordsInTitle', 'titleNo', 'wordNo', 'label'],
      dtype='object')

In [21]:
num_unique = {}
for attribute in attributes:
    num_unique[attribute] = len(pd.unique(df[attribute]))
num_unique

{'lineNo': 10936,
 'assgNo': 336,
 'fixcount': 8,
 'firstPassCnt': 7,
 'P1stFixation': 2,
 'P2stFixation': 2,
 'prevFixDur': 61,
 'firstfixDur': 63,
 'firstPassFixDur': 111,
 'nextFixDur': 68,
 'firstSaccLen': 9548,
 'lastSaccLen': 9350,
 'prevFixPos': 7866,
 'landingPos': 6847,
 'leavingPos': 6900,
 'totalFixDur': 149,
 'meanFixDur': 254,
 'nRegressFrom': 6,
 'regressLen': 572,
 'nextWordRegress': 2,
 'regressDur': 381,
 'pupilDiamMax': 3810,
 'pupilDiamLag': 2517,
 'timePrtctg': 1065,
 'nWordsInTitle': 9,
 'titleNo': 10,
 'wordNo': 10,
 'label': 3}

In [22]:
df2 = df.drop(columns=['lineNo','titleNo','wordNo'], axis=1) # Contains unique values for each instance, not going to be useful
# Also contants index values instead of actual data
df2.columns

Index(['assgNo', 'fixcount', 'firstPassCnt', 'P1stFixation', 'P2stFixation',
       'prevFixDur', 'firstfixDur', 'firstPassFixDur', 'nextFixDur',
       'firstSaccLen', 'lastSaccLen', 'prevFixPos', 'landingPos', 'leavingPos',
       'totalFixDur', 'meanFixDur', 'nRegressFrom', 'regressLen',
       'nextWordRegress', 'regressDur', 'pupilDiamMax', 'pupilDiamLag',
       'timePrtctg', 'nWordsInTitle', 'label'],
      dtype='object')

In [23]:
x = df2.drop(columns=['label'], axis=1)
y = df2['label']

In [24]:
# get the pearson correlation coefficients for all features to determine which features to train on

corr = df2.corr()
class_correlation = abs(corr["label"])
relevant_features = class_correlation[class_correlation > 0.1]
relevant_features

P2stFixation       0.154854
totalFixDur        0.125466
nRegressFrom       0.212710
nextWordRegress    0.228493
regressDur         0.214826
pupilDiamMax       0.122343
timePrtctg         0.168475
label              1.000000
Name: label, dtype: float64

In [25]:
# features should be independent of each other, test to make sure they aren't highly correlated with each other
feature_strings = ["P2stFixation", "totalFixDur", "nRegressFrom", "regressDur", "nextWordRegress", "pupilDiamMax", "timePrtctg"]  
for i in feature_strings:
    for j in feature_strings:
        if i != j:
            print(abs(df[[i,j]].corr()))
            print()

              P2stFixation  totalFixDur
P2stFixation      1.000000     0.087123
totalFixDur       0.087123     1.000000

              P2stFixation  nRegressFrom
P2stFixation      1.000000      0.250799
nRegressFrom      0.250799      1.000000

              P2stFixation  regressDur
P2stFixation      1.000000    0.251875
regressDur        0.251875    1.000000

                 P2stFixation  nextWordRegress
P2stFixation         1.000000         0.131331
nextWordRegress      0.131331         1.000000

              P2stFixation  pupilDiamMax
P2stFixation      1.000000      0.031823
pupilDiamMax      0.031823      1.000000

              P2stFixation  timePrtctg
P2stFixation      1.000000    0.048122
timePrtctg        0.048122    1.000000

              totalFixDur  P2stFixation
totalFixDur      1.000000      0.087123
P2stFixation     0.087123      1.000000

              totalFixDur  nRegressFrom
totalFixDur      1.000000      0.024128
nRegressFrom     0.024128      1.000000

           

In [26]:
# drop all columns which have a low correlation
# regressDur and timePtrctg have a high correlation with several other attributes and are thus not independent, drop both as well
final_columns = ["P2stFixation", "totalFixDur", "nRegressFrom", "nextWordRegress", "pupilDiamMax"]  
df_preprocessed_correlation = df2
for i in df2.columns:
    if i not in final_columns:
        df_preprocessed_correlation = df_preprocessed_correlation.drop(columns=[i,], axis=1)
        
df_preprocessed_correlation
    


,P2stFixation,totalFixDur,nRegressFrom,nextWordRegress,pupilDiamMax
0,0,100,0,0,0.0095
1,0,278,0,0,0.0095
2,0,159,0,0,0.0370
3,0,159,0,0,0.0370
4,0,139,0,0,0.0390
...,...,...,...,...,...
10931,0,139,0,1,0.4730
10932,0,219,1,0,0.4730
10933,0,99,0,1,0.4730
10934,0,358,1,0,0.2150


In [27]:
# perform attribute selection with RFE + decision trees
# RFE (Recursive Feature Elimination) feeds the data to a model, evaluates the performance for each attribute 
# and deletes attributes which don't perform well enough

rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
fit = rfe.fit(x, y)
best_features = []
print(len(fit.support_))
print(len(x.columns))
for i in range(len(fit.support_)):
    if fit.support_[i]:
        best_features.append(x.columns[i])
        
print(best_features)
df_preprocessed_rfe = df2

for i in df2.columns:
    if i not in best_features:
        df_preprocessed_rfe = df_preprocessed_rfe.drop(columns=[i,], axis=1)
        
df_preprocessed_rfe

24
24
['assgNo', 'firstSaccLen', 'lastSaccLen', 'pupilDiamMax', 'pupilDiamLag']


,assgNo,firstSaccLen,lastSaccLen,pupilDiamMax,pupilDiamLag
0,1,0.0000,382.8998,0.0095,0.145
1,1,87.0933,165.0068,0.0095,0.183
2,1,165.0068,141.1471,0.0370,0.183
3,1,141.1471,185.0007,0.0370,0.183
4,1,185.0007,221.6269,0.0390,0.183
...,...,...,...,...,...
10931,336,85.1469,194.6124,0.4730,0.069
10932,336,194.6124,224.5089,0.4730,0.069
10933,336,224.5089,229.9609,0.4730,0.473
10934,336,229.9609,0.0000,0.2150,0.215


In [28]:
df3 = df.sample(frac=1) # shuffles the dataframe
assert(len(df.index) == len(df3.index))

In [29]:
df3.head

<bound method NDFrame.head of       lineNo  assgNo  fixcount  firstPassCnt  P1stFixation  P2stFixation  \
3814    3815     122         1             1             1             0   
8302    8303     259         1             1             1             0   
6020    6021     178         1             1             1             0   
8941    8942     282         1             1             1             0   
8024    8025     249         1             1             1             1   
...      ...     ...       ...           ...           ...           ...   
2421    2422      76         1             1             1             0   
5884    5885     175         1             1             1             0   
2597    2598      78         1             1             0             1   
7326    7327     225         1             1             1             0   
467      468      14         1             1             1             1   

      prevFixDur  firstfixDur  firstPassFixDur  nextFixDu

In [47]:
freq_total = df3['label'].value_counts()
freq_total, freq_total[0]/sum(freq_total), freq_total[1]/sum(freq_total), freq_total[2]/sum(freq_total)

(1    4262
 0    3804
 2    2870
 Name: label, dtype: int64,
 0.3478419897585955,
 0.3897220190197513,
 0.26243599122165323)

In [35]:
X, y = df3.drop(columns=['label'], axis=1, inplace=False), df3['label']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, shuffle=True, stratify=y)

In [49]:
freq_total2 = y_train.value_counts()
freq_total2, freq_total2[0]/sum(freq_total2), freq_total2[1]/sum(freq_total2), freq_total2[2]/sum(freq_total2)

(1    3409
 0    3043
 2    2296
 Name: label, dtype: int64,
 0.3478509373571102,
 0.38968907178783724,
 0.26245999085505256)

In [50]:
freq_total3 = y_test.value_counts()
freq_total3, freq_total3[0]/sum(freq_total3), freq_total3[1]/sum(freq_total3), freq_total3[2]/sum(freq_total3)

(1    853
 0    761
 2    574
 Name: label, dtype: int64,
 0.34780621572212067,
 0.389853747714808,
 0.2623400365630713)

In [53]:
train_data = X_train.copy()
train_data['label'] = y_train.copy()
train_data.head

<bound method NDFrame.head of       lineNo  assgNo  fixcount  firstPassCnt  P1stFixation  P2stFixation  \
6593    6594     195         1             1             1             0   
5285    5286     159         1             1             1             0   
815      816      24         2             2             1             0   
9147    9148     287         1             1             1             0   
3084    3085      93         1             1             0             0   
...      ...     ...       ...           ...           ...           ...   
7146    7147     218         1             1             1             0   
1745    1746      54         1             2             1             0   
9329    9330     289         1             1             0             0   
8478    8479     267         1             1             1             0   
3481    3482     107         1             1             1             1   

      prevFixDur  firstfixDur  firstPassFixDur  nextFixDu

In [54]:
test_data = X_test.copy()
test_data['label'] = y_test.copy()

In [55]:
train_data.to_csv('train_data.csv')

In [56]:
test_data.to_csv('test_data.csv')